In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

cwd = '/content/drive/MyDrive/Colab Notebooks/data'

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!pip install "sacrebleu[ko]"

In [ ]:
import pandas as pd

train_df = pd.read_csv(os.path.join(cwd,'train.csv'))
train_df.head()

,id,text,mt,target
0,FAQEQ1Rt1ncRSxO,The winner of the first season of the show Gol...,쇼의 첫 번째 시즌의 우승자인 Golos Dina Garipova는 여배우 Irin...,쇼의 첫 번째 시즌의 우승자인 골로스디나 가리포바는 여배우 이리나 무라베바에게 헌정...
1,JSjY2hYQT4oPdz6,"Replica census entry for Gypsie Runningcloud, ...",Gypsie Runningcloud에 대한 복제 인구 조사 항목(48세).,"집시 런닝클라우드의 인구조사 자료 사본, 48세."
2,nykRvvrs3DjlWTQ,I don't think I've ever seen one where they in...,나는 그들이 대화를 포함하는 것을 본 적이 없다고 생각합니다.,저는 그것들이 대화를 포함하는 것을 본 적이 없다고 생각합니다.
3,yIzKFCfNBhLx2q5,The patients were not significantly slower on ...,"환자들은 회상, 반응 시간 또는 추론 테스트에서 유의하게 느려지지 않았습니다.","환자들은 회상, 반응 시간 또는 추론 테스트에서 유의하게 느려지지 않았습니다."
4,68Yvljqe5BS5CWQ,"The artist declares, despite her outward fragi...",그 예술가는 겉으로는 연약함에도 불구하고 엄격한 지도자라고 선언한다.,그 예술가는 겉으로는 연약함에도 불구하고 엄격한 지도자라고 선언한다.


In [ ]:
test_df = pd.read_csv(os.path.join(cwd,'test.csv'))
test_df.head()

,id,text
0,mcY54CiViEeLTeJ,The actress posted a photo of a brutal man in ...
1,aaUE07DPd3D4yU8,You learned about music.
2,ZmhbGbc4fuL6wWX,"When I bit into it, more than I expected, this..."
3,cuSNoRiXZm9ewP6,It's at a click of a button.
4,tqa1wfPjLzhDnE3,"It changes throughout childhood, but in adoles..."


In [ ]:
test_df.shape

(10757, 2)

In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("KES/T5-KES")

model = AutoModelForSeq2SeqLM.from_pretrained("KES/T5-KES")

text = "I am lived with my parenmts "
inputs = tokenizer("grammar:"+text, truncation=True, return_tensors='pt')

output = model.generate(inputs['input_ids'], num_beams=5, max_length=512, early_stopping=True)
correction=tokenizer.batch_decode(output, skip_special_tokens=True)
print("".join(correction)) #Correction: I am living with my parents.


I have lived with my parents.


In [ ]:
import torch


if torch.cuda.is_available():
    device = torch.device('cuda')
else:

    device = torch.device('cpu')

# 모델을 GPU로 이동
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
BATCH_SIZE = 32
preds = []


for i in range(0, len(test_df), BATCH_SIZE):

    batch_texts = [f"grammar: {text}" for text in test_df['text'][i:i+BATCH_SIZE]]

    inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)

    gen_seqs = model.generate(
        inputs.input_ids,
        num_beams=4,
        max_length=512,
    )

    batch_preds = tokenizer.batch_decode(gen_seqs, skip_special_tokens=True)
    preds.extend(batch_preds)


batch_df = pd.DataFrame({'id': test_df['id'], 'pred': preds})


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# load the model and tokenizer
model_path = "KETI-AIR-Downstream/long-ke-t5-base-translation-aihub-en2ko"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)



In [ ]:
import torch


if torch.cuda.is_available():
    device = torch.device('cuda')
else:

    device = torch.device('cpu')

# 모델을 GPU로 이동
model.to(device)

LongT5ForConditionalGeneration(
  (shared): Embedding(64100, 768)
  (encoder): LongT5Stack(
    (embed_tokens): Embedding(64100, 768)
    (block): ModuleList(
      (0): LongT5Block(
        (layer): ModuleList(
          (0): LongT5LayerTransientGlobalSelfAttention(
            (TransientGlobalSelfAttention): LongT5TransientGlobalAttention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
              (global_relative_attention_bias): Embedding(32, 12)
              (global_input_layer_norm): LongT5LayerNorm()
            )
            (layer_norm): LongT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): LongT5LayerFF(
            (DenseReluDense

In [ ]:
BATCH_SIZE = 32  # 원하는 배치 크기를 설정합니다. 이 값을 조정하여 가장 적합한 크기를 찾을 수 있습니다.
preds = []  # 예측을 저장할 리스트

# 배치별로 데이터를 처리합니다.
for i in range(0, len(batch_df), BATCH_SIZE):
    batch_sources = [f"translate_en2ko: {text}" for text in batch_df['pred'][i:i+BATCH_SIZE]]
    input_ids = tokenizer(batch_sources, padding=True, truncation=True, return_tensors="pt").input_ids.to(device)
    gen_seqs = model.generate(
        input_ids,
        num_beams=6,
        max_length=1024,

    )
    batch_preds = [tokenizer.decode(seq, skip_special_tokens=True) for seq in gen_seqs]
    preds.extend(batch_preds)

batch_df2 = pd.DataFrame({'id': test_df['id'], 'pred': preds})


/usr/local/lib/python3.10/dist-packages/transformers/models/longt5/modeling_longt5.py:170: UserWarning: An output with one or more elements was resized since it had shape [43], which does not match the required output shape [32, 43]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at ../aten/src/ATen/native/Resize.cpp:28.)
  true_block_ends = torch.logical_and(block_ends, block_ids >= 0)
/usr/local/lib/python3.10/dist-packages/transformers/models/longt5/modeling_longt5.py:146: UserWarning: An output with one or more elements was resized since it had shape [32, 1, 128, 1], which does not match the required output shape [32, 1, 128, 384]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tenso

In [ ]:
batch_df2.to_csv('prediction.csv', encoding='utf-8', index=False)